In [ ]:
from pydub import AudioSegment 
import time
from fastcore.all import *
from datetime import datetime

In [ ]:
def str2path(file):
    return file if isinstance(file, Path) else Path(file)

In [ ]:
def time2msec(time:str):
    pt = datetime.strptime(time,'%H:%M:%S' )
    return (pt.hour * 3600 + pt.minute * 60 + pt.second) * 1000

In [ ]:
def loadtsfile(file):
    file = str2path(file)
    with open(file,'r') as infile:
        ts = infile.readlines()
    tss = [x.strip('\n').split('\t')[0:3] for x in ts if len(x.split('\t'))>1]
    return [{'speaker':x[0], 'start':x[1], 'end':x[2]} for x in tss]


In [ ]:
def split_audio(
    input_file:Union[str,Path],
    ts_file:list[dict],
    output_folder:Union[str,Path] = None
):
    input_file = str2path(input_file)
    ts_file = str2path(ts_file)
    output_folder = str2path(output_folder) if output_folder else input_file.parent/input_file.stem
    
    if not output_folder.exists():
        output_folder.mkdir()

    
    ts_d = loadtsfile(ts_file)
    
    audio = AudioSegment.from_file(input_file)
    for i,limit in enumerate(ts_d):
        if limit['start'] == '0':
            limit['start'] = '0:0:0'
        clip = audio[time2msec(limit['start']):time2msec(limit['end'])]
        clip.export(output_folder/f"{limit['speaker']}_{i}.mp3", format='mp3')

In [ ]:
split_audio('Audios/Bolsonaro/Bolsonaro_1.mp3', 'Audios/Bolsonaro/ts_Bolsonaro_1.txt')

In [ ]:
with open('./Audios/Biden/ts_biden1.txt','r') as infile:
    ts = infile.readlines()

In [ ]:
tss = [x.strip('\n').split('\t')[0:3] for x in ts if len(x.split('\t'))>1]

In [ ]:
ts_d = [{'speaker':x[0], 'start':x[1], 'end':x[2]} for x in tss]


In [ ]:
song = AudioSegment.from_file("Audios/Biden/Biden_1.mp3", 
                              ) 

In [ ]:
for i,limit in enumerate(ts_d):
    if limit['start'] == '0':
        limit['start'] = '0:0:0'
    clip = song[time2msec(limit['start']):time2msec(limit['end'])]
    clip.export(f"{limit['speaker']}_{i}.mp3", format='mp3')
    